In [2]:
from model.types import MarketDemandSupply
from model.run import load_config, run, post_processing
import matplotlib.pyplot as plt
from model.mechanism.amm_k import amm_k_mechanism
from model.mechanism.treasury import liq_backing_mechanism,treasury_stables_mechanism,floating_supply_mechanism,liq_ohm_mechanism
from model.policy.rbs_price import lower_target_policy,upper_target_policy

params = {"demand_factor": [10],
         "supply_factor": [10],
         "initial_reserves_volatile":[17000000], # setting it as constant since in this simulation we don't use it at all.
         "max_liq_ratio":[0.14375],  # LiquidityUSD : reservesUSD ratio --> 1:1 = 0.5
         "target_ma":[30], # number of days
         "lower_wall":[0.15],
         "upper_wall":[0.15],
         "lower_cushion":[.075],
         "upper_cushion":[.075],
          "reinstate_window": [30],
          "max_outflow_rate": [0.05],
          "reward_rate_policy": ["Flat"],
          "ask_factor": [0.095],  # % of floating supply that the treasury can deploy when price is trading above the upper target
          "bid_factor": [0.095],  # % of the reserves that the treasury can deploy when price is trading below the lower target
    "cushion_factor": [0.3075],  # The percentage of a bid or ask to offer as a cushion
        "min_counter_reinstate": [6],  # Number of days within the reinstate window that conditions are true to reinstate a bid or ask
         "with_reinstate_window": ['Yes']
         }


initialstate = { # variable values borrowed from liquidity-olympus/simulation.ipynb
            'supply':30000000,
            "liq_stables": 21000000,
            "reserves_stables":200000000,
            "reserves_volatile":17000000,
            "price":9,
            "reward_rate": 0.000198,

            "market_demand_supply": MarketDemandSupply(total_supply = 50,
                                                   total_demand = 50),
                                                   
            # treasury policy
            'target_liq_ratio_reached':False,
            'reserves_in':0,
            'supply':25000000,

            # RBS related
            'ma_target':9,
            # market transaction variables
            'ask_change_ohm':0,
            'bid_change_ohm':0,
            "net_flow": None,
    "bid_capacity_target": None,
    "ask_capacity_target": None,
    "bid_capacity_target_cushion": None,
    "ask_capacity_target_cushion": None,
    "natural_price": None,
    "bid_capacity_cushion": None,
    "ask_capacity_cushion": None,
    "bid_change_cushion_usd": None,
    "bid_change_cushion_ohm": None,
    "ask_change_cushion_usd": None,
    "ask_change_cushion_ohm": None,
    "bid_change_usd": None,
    
    
            }
# initialize parameters that are decided by other parameters
initialstate['amm_k'] = amm_k_mechanism(initialstate)
initialstate['price_history']=[initialstate['price']]

initialstate['lb_target'] = initialstate['ma_target']
initialstate['price_target']=initialstate['ma_target']

initialstate['liq_ohm'] = liq_ohm_mechanism(initialstate['liq_stables'],initialstate['price'])
initialstate['floating_supply'] = floating_supply_mechanism(initialstate['supply'],initialstate['liq_ohm'])



initialstate['treasury_stables'] = treasury_stables_mechanism(initialstate['liq_stables'],initialstate['reserves_stables'])
initialstate['liq_backing'] = liq_backing_mechanism(initialstate['treasury_stables'],params['initial_reserves_volatile'][0])
initialstate['lower_target_wall'] = lower_target_policy(initialstate['price_target'], params['lower_wall'][0])
initialstate['upper_target_wall'] = upper_target_policy(initialstate['price_target'], params['upper_wall'][0])
initialstate["lower_target_cushion"] = lower_target_policy(initialstate['price_target'], params['lower_cushion'][0])
initialstate["upper_target_cushion"] = upper_target_policy(initialstate['price_target'], params['upper_cushion'][0])

initialstate['bid_counter'],initialstate['ask_counter'] = [0] * params["reinstate_window"][0],[0]* params["reinstate_window"][0]


initialstate["bid_capacity_target"] = params["bid_factor"][0] * initialstate["reserves_stables"]
initialstate["ask_capacity_target"] = params["ask_factor"][0] * initialstate["reserves_stables"] / initialstate["upper_target_wall"] * (1 + params["lower_wall"][0] + params["upper_wall"][0])
initialstate["bid_capacity_target_cushion"] = initialstate["bid_capacity_target"] * params["cushion_factor"][0]
initialstate["ask_capacity_target_cushion"] = initialstate["ask_capacity_target"] * params["cushion_factor"][0]
initialstate["bid_capacity"] = initialstate["bid_capacity_target"]
initialstate["ask_capacity"] = initialstate["ask_capacity_target"]
initialstate["bid_capacity_cushion"] = initialstate["bid_capacity_target_cushion"]
initialstate["ask_capacity_cushion"] = initialstate["ask_capacity_target_cushion"]
initialstate["ask_change_ohm"] = 0
initialstate["bid_change_ohm"] = 0


nrun = 2
nstep = 365
exp = load_config(nrun, params, initialstate, nstep)
raw = run(exp)
df = post_processing(raw)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 365, 17, 2, 41)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (365, 17, 2, 41)
Execution Method: local_simulations
Execution Mode: parallelized


KeyError: 'ask_change_usd'

In [ ]:
for col in ["supply", "reserves_in", "ma_target", "lb_target",
           "control_ask", "control_bid",
           "bid_capacity_target", "ask_capacity_target", "bid_capacity_target_cushion",
 "ask_capacity_target_cushion", "natural_price"]:
    df.pivot("timestep","run", col).plot(kind='line')
    plt.title(col)
    plt.show()

In [ ]:
for col in ["bid_capacity_cushion", "ask_capacity_cushion"]:
    df.pivot("timestep","run", col).plot(kind='line')
    plt.title(col)
    plt.show()

In [ ]:
for col in ["bid_change_cushion_usd", "bid_change_cushion_ohm", "ask_change_cushion_usd", "ask_change_cushion_ohm"]:
    df.pivot("timestep","run", col).plot(kind='line')
    plt.title(col)
    plt.show()